In [4]:
with open(
    "/Users/apple/Desktop/workspace/UsefulTool/exp-command-generator/test.txt", 'r'
) as f:
    contents = f.read()

In [5]:
# find all "#####" indexes
import re
indexes = [m.start() for m in re.finditer('#####', contents)]

In [6]:
assert len(indexes) % 2 == 0

In [7]:
# split to span
spans = []
# spans.append(contents[:indexes[0]])
for i in range(len(indexes)):
    if i != len(indexes) - 1:
        spans.append(contents[indexes[i]:indexes[i+1]])
# spans.append(contents[indexes[-1]:])

In [8]:
spans_with_type = [
    
]
for span in spans:
    if "setup" in span:
        spans_with_type.append((span, "setup"))
    elif "loop" in span:
        spans_with_type.append((span, "loop"))
    elif "main" in span:
        spans_with_type.append((span, "command"))
    else:
        spans_with_type.append((span, "other"))
    

In [9]:
spans_with_type

[('##### setup\n#!/bin/bash\nfile_name=$(basename $0)\ncurrent_path=$(pwd)\ncd /data/yixin/workspace/unl-graph-usenix\nsource activate /data/yixin/anaconda/unlg\ndatasets=("IMDB-BINARY"  "MUTAG" "ENZYMES" "IMDB-MULTI"  )\nmodels=( "gcn" "gin"  "sage" )\nentity="mib-nlp"\nexp_name="adv-run-v3"\nbatch_size=8\nmethods=( "clean" "rand"  "feat" "grad" "inject"  "adv")\nwd=1e-5\nadv_train_budgets=( 0.07 0.09 0.11 )\ngen_exp_name="main-results-v2"\nlr=0.01\nes_patience=40\nseed_default=0\noptimizer="adam"\nbudget=0.05\ntotal_epoch=300\nmax_steps=5000\nseeds=("402")\n# mkdir $current_path/logs/ if not exist\nmkdir -p $current_path/logs/\nmkdir -p $current_path/logs/$exp_name\n',
  'setup'),
 ('#####\n\n\n\n', 'other'),
 ('##### loop\nfor adv_train_budget in "${adv_train_budgets[@]}"; do\nfor dataset in "${datasets[@]}"; do\nfor model in "${models[@]}"; do\nfor method in "${methods[@]}"; do\n',
  'loop'),
 ('##### \n\n    ', 'other'),
 ('##### main\n    comb_command="for seed in ${seeds[@]} ; d

In [10]:
gpu_env = """
username_mine=root
max_gpu_utilization=90
total_aviable=24564
max_gpu_memory_gap=5000
available_devices=( 0 1 2 3 4 5 6 7 8 9 )
current_device_idx=-1
sleeptime=30
cpu_mean_max=77
memory_rate_max=80
constrain_total=true
constrain_mine=false
constrain_rate=2
"""

In [79]:
update_device_func = """
function update_device_idx {
    sleep $sleeptime
    if [ $constrain_total = true ]; then
    # check total cpu usage
    while true; do
        cpu_mean_1=$(mpstat -P ALL 1 1 | awk '/Average:/ && $2 ~ /[0-9]/ { cpu_usage=100-$NF; total+=cpu_usage; count++ } END { print total/count }')
        sleep 1
        cpu_mean_2=$(mpstat -P ALL 1 1 | awk '/Average:/ && $2 ~ /[0-9]/ { cpu_usage=100-$NF; total+=cpu_usage; count++ } END { print total/count }')
        sleep 1
        cpu_mean_3=$(mpstat -P ALL 1 1 | awk '/Average:/ && $2 ~ /[0-9]/ { cpu_usage=100-$NF; total+=cpu_usage; count++ } END { print total/count }')
        cpu_mean=$(echo "scale=2; ($cpu_mean_1+$cpu_mean_2+$cpu_mean_3)/3" | bc)

        # if currently cpu usage is less than the threshold, then break
        if [ $(echo "$cpu_mean < $cpu_mean_max" | bc) -eq 1 ]; then
            echo "total cpu mean: $cpu_mean is less than $cpu_mean_max, continue to check total memory usage"
            break
        else
            echo "total cpu mean: $cpu_mean is greater than $cpu_mean_max, sleep 10 seconds"
            sleep 10
        fi
    done;

    # check total memory usage
    while true; do
        # get memory usage of whole system
        mem_used_1=$(free -m | awk '/Mem:/ {print $3}')
        sleep 1
        mem_used_2=$(free -m | awk '/Mem:/ {print $3}')
        sleep 1
        mem_used_3=$(free -m | awk '/Mem:/ {print $3}')
        mem_used=$(echo "scale=2; ($mem_used_1+$mem_used_2+$mem_used_3)/3" | bc)
        
        # echo $mem_used
        # get rate of memory usage
        mem_rate=$(echo "scale=2; $mem_used/$(free -m | awk '/Mem:/ {print $2}')*100" | bc)
        # echo $mem_rate
        if [ $(echo "$mem_rate < $memory_rate_max" | bc) -eq 1 ]; then
            echo "total memory rate: $mem_rate is less than $memory_rate_max, continue to check my own cpu and memory usage"
            break
        else
            echo "total memory rate: $mem_rate is greater than $memory_rate_max, sleep 10 seconds"
            sleep 10
        fi
    done;
    fi;

    # if constrain_mine
    if [ $constrain_mine = true ]; then

        # check my own cpu and memory usage, it should be less than 1/$constrain_rate of the given cpu_mean_max / memory_rate_max
        while true; do
            username=$username_mine
            cpu_usage_user_sum=$(ps -u $username -o %cpu | awk '{sum+=$1} END {print sum}')
            # echo $cpu_usage_user_sum
            total_aviable_cpu=$(nproc)
            total_aviable_cpu=$(echo "$total_aviable_cpu*100" | bc)
            # echo $total_aviable_cpu
            cpu_usage_user_ratio=$(echo "scale=2; $cpu_usage_user_sum/$total_aviable_cpu*100" | bc)
            # echo $cpu_usage_user_ratio

            memory_usage_user_sum=$(ps -u $username -o rss | awk '{sum+=$1} END {print sum/1024}')
            # echo $memory_usage_user_sum
            memory_usage_total=$(free -m | awk '/Mem:/ {print $2}')
            # echo $memory_usage_total
            memory_usage_user_ratio=$(echo "scale=2; $memory_usage_user_sum/$memory_usage_total*100" | bc)
            # echo $memory_usage_user_ratio

            # so my ratio should be less than 1/$constrain_rate of the given threshold
            cpu_mean_max_mine=$(echo "$cpu_mean_max/$constrain_rate" | bc)
            memory_rate_max_mine=$(echo "$memory_rate_max/$constrain_rate" | bc)
            if [ $(echo "$cpu_usage_user_ratio < $cpu_mean_max_mine" | bc) -eq 1 ] && [ $(echo "$memory_usage_user_ratio < $memory_rate_max_mine" | bc) -eq 1 ]; then
                echo "my cpu usage: $cpu_usage_user_ratio, memory usage: $memory_usage_user_ratio is less than half of the given threshold for cpu: $cpu_mean_max_mine and memory: $memory_rate_max_mine, ready to take off"
                break
            else
                echo "my cpu usage: $cpu_usage_user_ratio, memory usage: $memory_usage_user_ratio is greater than half of the given threshold, sleep 10 seconds"
                sleep 10
            fi
        done;
    fi;

    # so all the conditions are satisfied, we can update the device idx and run the next experiment
    while true; do
        current_device_idx=$((current_device_idx+1))
        if [ $current_device_idx -ge ${#available_devices[@]} ]; then
            # reset 
            current_device_idx=0
        fi
        # check whether this device is fully booked using nvidia-smi
        # get the gpu current memory usage 
        useage=$(nvidia-smi --query-gpu=memory.used --format=csv,noheader,nounits -i ${available_devices[$current_device_idx]})
        utilization=$(nvidia-smi --query-gpu=utilization.gpu --format=csv,noheader,nounits -i ${available_devices[$current_device_idx]})
        
        if [ $useage -ge $((total_aviable-max_gpu_memory_gap)) ] || [ $utilization -ge $max_gpu_utilization ]; then
            echo "device ${available_devices[$current_device_idx]} is fully booked, try next one"
            sleep 3
            continue
        else
            break
        fi
    done
    echo "current device: ${available_devices[$current_device_idx]}"
    device=${available_devices[$current_device_idx]}
}
"""

In [80]:
update_device_command = "update_device_idx;\n"

In [81]:
backend_run = False

In [82]:
gpu_utility = ""
gpu_utility = gpu_env + "\n\n" + update_device_func 
with open("gpu_utility.sh", 'w') as f:
    f.write(gpu_utility)

In [83]:
spans_with_type_added_device_control = []

for span, type_ in spans_with_type:
    if type_ == "setup":
        spans_with_type_added_device_control.append((
            """cd $(cd "$(dirname "$0")";pwd); source gpu_utility.sh\n\n"""
            , "device_control"))
        spans_with_type_added_device_control.append((span, type_))
        # spans_with_type_added_device_control.append((gpu_env, "device_control"))
        # spans_with_type_added_device_control.append((update_device_func, "device_control"))
    elif type_ == "loop":
        spans_with_type_added_device_control.append((span, type_))
    elif type_ == "command":
        spans_with_type_added_device_control.append(("\n"+update_device_command, "device_control"))
        span_remove_the_first_part = span[span.index("\n"):]
        spans_with_type_added_device_control.append((f"\n\ncommand=\"\"\"{span_remove_the_first_part}\"\"\"\n", type_))
        run_command = "eval $command"
        if backend_run:
            run_command += " &"
        run_command += "\n\n\n"
        spans_with_type_added_device_control.append((run_command, type_))
    else:
        spans_with_type_added_device_control.append((span, type_))
spans_without_type = [span for span, type_ in spans_with_type_added_device_control]
spans_without_type_str = "".join(spans_without_type)
with open("./output.sh", 'w') as f:
    f.write(spans_without_type_str)